In [1]:
import cv2
import numpy as np

# --- Calibration Constant ---
# You need to calibrate this once (see bottom of previous response).
# Default approx value:
FOCAL_LENGTH = 728  

def nothing(x):
    pass

def main():
    cap = cv2.VideoCapture(1)
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 360)

    # Create Window
    cv2.namedWindow("Universal Detector")

    # --- Sliders (Trackbars) ---
    # 1. Edge Sensitivity: Adjusts how "strict" the edge detection is
    cv2.createTrackbar("Threshold", "Universal Detector", 100, 255, nothing)
    
    # 2. Known Width: You must tell the code how wide your object is (in inches)
    # Default set to 4 inches (approx width of a phone)
    cv2.createTrackbar("Object Width (in)", "Universal Detector", 4, 20, nothing)

    print("Press 'q' to quit.")

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # 1. Convert to Grayscale (Edge detection works on brightness, not color)
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # 2. Blur to reduce noise (Important for clean edges)
        blurred = cv2.GaussianBlur(gray, (5, 5), 0)

        # 3. Get Threshold from Slider
        thresh_val = cv2.getTrackbarPos("Threshold", "Universal Detector")
        
        # 4. Canny Edge Detection
        # Finds edges based on the threshold
        edges = cv2.Canny(blurred, thresh_val, thresh_val * 2)

        # Dilate edges to close gaps (makes the outline continuous)
        kernel = np.ones((3, 3), np.uint8)
        edges = cv2.dilate(edges, kernel, iterations=1)

        # 5. Find Contours
        contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        if contours:
            # Sort contours by area and get the largest one
            # This assumes the object you want is the biggest thing in the frame
            largest_contour = max(contours, key=cv2.contourArea)

            # Filter small noise (Must be bigger than 1000 pixels area)
            if cv2.contourArea(largest_contour) > 1000:
                # Get Bounding Box
                x, y, w, h = cv2.boundingRect(largest_contour)

                # --- Distance Math ---
                # Get the "Real Width" from the slider
                known_width = cv2.getTrackbarPos("Object Width (in)", "Universal Detector")
                
                if known_width > 0 and w > 0:
                    # Formula: Distance = (Real_Width * Focal_Length) / Pixel_Width
                    distance = (known_width * FOCAL_LENGTH) / w

                    # --- Visuals ---
                    # Draw Green Box
                    cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
                    
                    # Draw Red "Centroid" Dot
                    cx = x + w // 2
                    cy = y + h // 2
                    cv2.circle(frame, (cx, cy), 5, (0, 0, 255), -1)

                    # Text
                    cv2.putText(frame, f"Dist: {distance:.2f} inches", (x, y - 10),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
                    cv2.putText(frame, f"Obj Width: {known_width} in", (10, 30),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 255), 2)

        # Show the normal video
        cv2.imshow("Universal Detector", frame)
        
        # OPTIONAL: Show the "Edges" view to see what the computer sees
        # cv2.imshow("Edges View", edges)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()

Press 'q' to quit.
